In [11]:
import json
from pprint import pprint
import matplotlib as mpl
import matplotlib.pyplot as pltfrom 

from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np

In [27]:
with open("Headline_Trainingdata.json") as f:
    reviews_train = json.load(f)

with open("Headline_Trialdata.json") as f:
    reviews_trial = json.load(f)

with open("Headlines_Testdata.json", encoding="utf8") as f:
    reviews_test = json.load(f)

maxlen = 0
#pprint(reviews_train)
#X_train = [review['title'] for review in reviews_train]
X_trial = [review['title'] for review in reviews_trial]
#Y_train = [0 if review['sentiment']<=0 else 1 for review in reviews_train]
Y_trial = [0 if review['sentiment']<=0 else 1 for review in reviews_trial]

X_train = []
X_test = [review['title'] for review in reviews_test]
Y_train = []
Y_test = []
for review in reviews_train :
    X_train.append(review['title'])
    Y_train.append(0 if review['sentiment']<=0 else 1)
    if maxlen < len(review['title'].split()) :
        maxlen = len(review['title'].split())
    
for review in reviews_trial :
    X_train.append(review['title'])
    Y_train.append(0 if review['sentiment']<=0 else 1)
    if maxlen < len(review['title'].split()) :
        maxlen = len(review['title'].split())

testMax = 0
for review in reviews_test :
    if testMax < len(review['title'].split()) :
        testMax = len(review['title'].split())
print(len(X_train)+len(X_test), len(Y_train))
print(testMax)


1647 1156
18


In [28]:
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
data = pad_sequences(sequences, maxlen=50)

In [29]:
model = Sequential()
model.add(Embedding(15000, 16, input_length=50))
model.add(LSTM(16, dropout=0.15, recurrent_dropout=0.15))
#model.add(Dense(4, activation='tanh'))
model.add(Dense(1, activation='tanh'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 50, 16)            240000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 16)                2112      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 17        
Total params: 242,129
Trainable params: 242,129
Non-trainable params: 0
_________________________________________________________________


In [30]:


history = model.fit(data, np.array(Y_train), validation_split=0.4, epochs=75,verbose = 1)

#plot_training_curves(history.history);

Train on 693 samples, validate on 463 samples
Epoch 1/75
693/693 [==============================] - 4s - loss: 2.0875 - acc: 0.4214 - val_loss: 1.3044 - val_acc: 0.4449
Epoch 2/75
693/693 [==============================] - 1s - loss: 1.1652 - acc: 0.4214 - val_loss: 0.9717 - val_acc: 0.4449
Epoch 3/75
693/693 [==============================] - 1s - loss: 0.8217 - acc: 0.4416 - val_loss: 0.6743 - val_acc: 0.5572
Epoch 4/75
693/693 [==============================] - 1s - loss: 0.6671 - acc: 0.5945 - val_loss: 0.6760 - val_acc: 0.6156
Epoch 5/75
693/693 [==============================] - 1s - loss: 0.6264 - acc: 0.6335 - val_loss: 0.6564 - val_acc: 0.6674
Epoch 6/75
693/693 [==============================] - 1s - loss: 0.5592 - acc: 0.7388 - val_loss: 0.6372 - val_acc: 0.6566
Epoch 7/75
693/693 [==============================] - 1s - loss: 0.5089 - acc: 0.8081 - val_loss: 0.6183 - val_acc: 0.6847
Epoch 8/75
693/693 [==============================] - 1s - loss: 0.4391 - acc: 0.8470 - val_l

693/693 [==============================] - 1s - loss: 0.0196 - acc: 0.9134 - val_loss: 1.4179 - val_acc: 0.6998
Epoch 67/75
693/693 [==============================] - 1s - loss: 0.0213 - acc: 0.9235 - val_loss: 1.4235 - val_acc: 0.6976
Epoch 68/75
693/693 [==============================] - 1s - loss: 0.0198 - acc: 0.9149 - val_loss: 1.4142 - val_acc: 0.6911
Epoch 69/75
693/693 [==============================] - 1s - loss: 0.0176 - acc: 0.9380 - val_loss: 1.4228 - val_acc: 0.6847
Epoch 70/75
693/693 [==============================] - 1s - loss: 0.0176 - acc: 0.9365 - val_loss: 1.4238 - val_acc: 0.6976
Epoch 71/75
693/693 [==============================] - 1s - loss: 0.0185 - acc: 0.9192 - val_loss: 1.4061 - val_acc: 0.6955
Epoch 72/75
693/693 [==============================] - 1s - loss: 0.0193 - acc: 0.9076 - val_loss: 1.4251 - val_acc: 0.6976
Epoch 73/75
693/693 [==============================] - 1s - loss: 0.0185 - acc: 0.9134 - val_loss: 1.4281 - val_acc: 0.7019
Epoch 74/75
693/693 

In [33]:
sequences = tokenizer.texts_to_sequences(np.array(X_trial))
data = pad_sequences(sequences, maxlen=50)
predictions = model.predict(data)
for i in range(len(predictions)):
    print(reviews_trial[i]['title'],reviews_trial[i]['sentiment'],"%.3f" % predictions[i])

Tesco says UK store closures put 2000 jobs at risk -0.9 -0.436
CRH's concrete bid for Holcim Lafarge assets 0.3 1.000
CRH's concrete bid for Holcim Lafarge assets 0.3 1.000
Reed Elsevier share price slides on underwhelming full-year results -0.9 0.299
Kingfisher bid for Mr Bricolage runs into trouble -0.3 -0.513
Kingfisher bid for Mr Bricolage runs into trouble -0.3 -0.513
Glencore's annual results beat forecasts 0.9 0.446
Diageo stays neutral on India boardroom turmoil -0.2 -0.516
Shell to Cut 6500 Jobs as Profit Drops -1 0.438
Markets Shire up 2.5% and Baxalta up 6% on $32bn deal 0.8 1.000
Markets Shire up 2.5% and Baxalta up 6% on $32bn deal 0.8 1.000
BP ends 27-year sponsorship of Tate as falling oil price takes toll -0.2 -0.565
HSBC, Standard Chartered Lead Asia Bank Rout as U.K. Votes 'Out' -1 -0.139
HSBC, Standard Chartered Lead Asia Bank Rout as U.K. Votes 'Out' -1 -0.139
